import camelot
import pandas as pd
import numpy as np
import tabula
import re

In [12]:
##this is visual studio


In [10]:
x=camelot.read_pdf("file:///C:/Users/hp/Downloads/BTECH21-Grade-sheet.pdf",pages='all',flavor='stream',edge_tol=1,columns=['2']*8)

In [11]:
len(x)

143

In [5]:
def row(roll_index,index,x):
    roll=re.search("\d\dVV\dA\d\d\d\d",x.df.iloc[roll_index,1].splitlines()[0]).group()
    l=x.df.iloc[index,1].splitlines()
    l.append(roll)
    l[4]=l[4][0]
    y=np.reshape(l,(1,6))
    return [y,roll_index,index+1]
#l,b,c=row(roll_index,index)

In [6]:
df=pd.DataFrame()
def f(x):
    roll_index=3
    index=roll_index+2
    df=pd.DataFrame(columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])
    while(index<=len(x.df.iloc[:,1])):
        z=x.df.iloc[index,1].splitlines()[0]
        y="TOTAL"
        i=0
        while(z!=y):
            #print(1)
            l,roll_index,index=row(roll_index,index,x)
            a=pd.DataFrame(l,columns=['NO', 'SUBCODE', 'SUBNAME', 'GRADE', 'CREDITS',"roll_no"])
            df=df.append(a)
            #print(df.iloc[:,:])
            i=i+1
            z=x.df.iloc[index,1].splitlines()[0]
        roll_index=index+4
        index=roll_index+2
        #print(df)
    return pd.DataFrame(df)
for i in range(len(x)):
    df=df.append(f(x[i]))

In [7]:
df.rename(columns={'SUBNAME':'Subname','GRADE':'Grade','CREDITS':'Credits','roll_no':'Htno','NO':'Sno'},inplace=True)
df.drop('SUBCODE',axis=1,inplace=True)
df.set_index('Sno',inplace=True)

In [8]:
df['Credits']=df['Credits'].astype('int')

In [9]:
df.head()

,Subname,Grade,Credits,Htno
Sno,,,,
1,PROBABILITY & STATISTICS,F,0,16VV1A1203
2,MATHEMATICAL FOUNDATIONS OF COMPUTER SCIENCE,F,0,16VV1A1203
3,DATA STRUCTURES THOROUGH C++,F,0,16VV1A1203
4,DIGITAL LOGIC DESIGN,F,0,16VV1A1203
5,PYTHON PROGRAMMING,D,3,16VV1A1203


In [10]:
#chainging to requried format
df['points']=df['Grade'].map({'O':10,'S':9,'A':8,'B':7,'C':6,'D':5,'F':0,"ABSENT":0})
#df['Credits']=df['Credits'].apply(lambda x:int(str(x).split(" ")[0]))
df['Total']=df['points']*df['Credits']
#df.drop(['Grade','points'],axis=1,inplace=True)
df['sep']=df['Htno'].apply(lambda x:x[6:8])

In [11]:
#division based on roll numbers
unique=df['sep'].unique()
seperate=[]#joining the rows with same Htno
for i in range(len(unique)):
    seperate.append(df[df.sep==unique[i]])

In [12]:
seperate[0]

,Subname,Grade,Credits,Htno,points,Total,sep
Sno,,,,,,,
1,PROBABILITY & STATISTICS,F,0,16VV1A1203,0,0,12
2,MATHEMATICAL FOUNDATIONS OF COMPUTER SCIENCE,F,0,16VV1A1203,0,0,12
3,DATA STRUCTURES THOROUGH C++,F,0,16VV1A1203,0,0,12
4,DIGITAL LOGIC DESIGN,F,0,16VV1A1203,0,0,12
5,PYTHON PROGRAMMING,D,3,16VV1A1203,5,15,12
...,...,...,...,...,...,...,...
4,DIGITAL LOGIC DESIGN,C,3,19VV5A1275,6,18,12
5,PYTHON PROGRAMMING,B,3,19VV5A1275,7,21,12
6,FORMAL LANGUAGES & AUTOMATA THEORY,A,3,19VV5A1275,8,24,12


In [13]:
#requried format
table=[]
for i in range(len(seperate)):
    table.append(seperate[i].pivot_table(index='Htno',columns='Subname',values='points',fill_value=0))
    table[i]['Credits']=seperate[i].groupby('Htno')['Credits'].sum()
    table[i]["SGPA"]=seperate[i].groupby('Htno')['Total'].sum()/table[i]['Credits']

In [14]:
grade={10:'O',9:'S',8:'A',7:'B',6:'C',5:'D',0:'F'}
for i in range(len(table)):
    col=list(table[i].columns).index('Credits')
    table[i].iloc[:,:col]=table[i].iloc[:,:col].replace(grade)
    table[i].reset_index(inplace=True)
    table[i].index = np.arange(1, len(table[i]) + 1)
    table[i]=table[i].rename_axis('SNo',axis=1)

In [148]:
#total and SGPA calculation
''''for i in range(len(table)):
    table[i]['Total']=table[int(i)].iloc[:,:-1].sum(axis=1)
    table[i]['SGPA']=(table[i]['Total']/table[i]['Credits'])
    table[i].reset_index(inplace=True)
    table[i].index = np.arange(1, len(table[i]) + 1)
    table[i]=table[i].rename_axis('SNo',axis=1)'''''

"'for i in range(len(table)):\n    table[i]['Total']=table[int(i)].iloc[:,:-1].sum(axis=1)\n    table[i]['SGPA']=(table[i]['Total']/table[i]['Credits'])\n    table[i].reset_index(inplace=True)\n    table[i].index = np.arange(1, len(table[i]) + 1)\n    table[i]=table[i].rename_axis('SNo',axis=1)"

In [184]:
len(table[5][table[5]['PROBABILITY & STATISTICS']=='B'])

25

In [189]:
list((table[5]=='B').sum())[1:-2]

c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


[19, 14, 5, 12, 12, 25, 24, 0]

In [15]:
#analysis
an=[]
pp=[]
for i in range(len(table)):
    n=list(table[i].columns).index('Htno')
    m=list(table[i].columns).index('Credits')
    analysis={
    'O grade':list((table[i]=='O').sum())[1:-2],
    'S grade':list((table[i]=='S').sum())[1:-2],
    'A grade':list((table[i]=='A').sum())[1:-2],
    'B grade':list((table[i]=='B').sum())[1:-2],
    'C grade':list((table[i]=='C').sum())[1:-2],
    'D grade':list((table[i]=='D').sum())[1:-2],
    'Fail':list((table[i]=='F').sum())[1:-2]
    }
    c=pd.DataFrame(analysis)
    c=c.T
    #print(n,m)
    c.columns=(table[i].columns)[n+1:m]
    an.append(c)
    pa={'pass':[len(table[i][table[i]['Credits']==22]['Credits'])],
     'pass per':100-int((len(table[i][table[i]['Credits']<22]['Credits'])/len(table[i]))*100)
    }
    d=pd.DataFrame(pa)
    d=d.T
    pp.append(d)

c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform eleme

c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform eleme

In [198]:
'''''import matplotlib.pyplot as plt
for i in range(len(table)):
    n=list(table[i].columns).index('Htno')
    m=list(table[i].columns).index('Credits')
    subjects=list(table[i].columns)[n+1:m]
    labels=list(an[i].index)
    for j in range(len(labels)):
        values=an[i][subjects[j]]
        fig1, ax1 = plt.subplots()
        ax1.pie(values, autopct='%1.1f%%',
        shadow=True, startangle=90)
        ax1.axis('equal')
        plt.legend(labels,loc='best')
        plt.title(subjects[j])
        plt.show()
        break'''''

"''import matplotlib.pyplot as plt\nfor i in range(len(table)):\n    n=list(table[i].columns).index('Htno')\n    m=list(table[i].columns).index('Credits')\n    subjects=list(table[i].columns)[n+1:m]\n    labels=list(an[i].index)\n    for j in range(len(labels)):\n        values=an[i][subjects[j]]\n        fig1, ax1 = plt.subplots()\n        ax1.pie(values, autopct='%1.1f%%',\n        shadow=True, startangle=90)\n        ax1.axis('equal')\n        plt.legend(labels,loc='best')\n        plt.title(subjects[j])\n        plt.show()\n        break"

In [16]:
#colour cells with 0 values 'fail'
styler_obj=[]
for i in range(len(table)):
    styler_obj.append(table[i].style.applymap(lambda x: 'background-color : #FFFF99' if x=='F' else ''))

In [ ]:
seperate[0]

,Subname,Grade,Credits,Htno,points,Total,sep
Sno,,,,,,,
1,PROBABILITY & STATISTICS,F,0,16VV1A1203,0,0,12
2,MATHEMATICAL FOUNDATIONS OF COMPUTER SCIENCE,F,0,16VV1A1203,0,0,12
3,DATA STRUCTURES THOROUGH C++,F,0,16VV1A1203,0,0,12
4,DIGITAL LOGIC DESIGN,F,0,16VV1A1203,0,0,12
5,PYTHON PROGRAMMING,D,3,16VV1A1203,5,15,12
...,...,...,...,...,...,...,...
4,DIGITAL LOGIC DESIGN,C,3,19VV5A1275,6,18,12
5,PYTHON PROGRAMMING,B,3,19VV5A1275,7,21,12
6,FORMAL LANGUAGES & AUTOMATA THEORY,A,3,19VV5A1275,8,24,12


In [17]:
#writing to excel
for i in range(len(table)):
    writer = pd.ExcelWriter('grades'+str(i)+'.xlsx', engine='xlsxwriter')
    styler_obj[i].to_excel(writer,sheet_name='Sheet'+str(i))
    an[i].to_excel(writer,sheet_name='Sheet'+str(i),startrow=len(table[i])+2,index=True)
    pp[i].to_excel(writer,sheet_name='Sheet'+str(i), startrow=len(table[i])+len(an[i])+5,index=True,header=None)
    
    for j in range(1,len(an[i].columns)+1):
        dt=table[i].iloc[:,[0,j]]
        dt.to_excel(writer,sheet_name='Sheet'+str(i+j))
        c=pd.DataFrame(an[i].iloc[:,j-1])
        c.to_excel(writer,sheet_name='Sheet'+str(i+j),startrow=len(dt)+1,index=True)
        workbook=writer.book
        worksheet = writer.sheets['Sheet'+str(i+j)]
        chart= workbook.add_chart({'type': 'pie'})
        chart.set_title({'name':(an[i].columns)[j-1]})
        chart.add_series({
        'categories':['Sheet'+str(i+j), len(table[i])+2, 0,len(table[i])+len(an[i])+1, 0],
        'values': ['Sheet'+str(i+j), len(table[i])+2, 1, len(table[i])+len(an[i])+1, 1],
        })
        worksheet.insert_chart('D10', chart)
    writer.save()